# Biostats 521: Lab 1
Brian Kim
2023-09-22

In [2]:
# Import Packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from scipy.stats import skew
from scipy.stats import ttest_ind
from scipy.stats import ks_2samp
from scipy.stats import mannwhitneyu
from statsmodels.graphics.gofplots import qqplot
from statsmodels.stats.diagnostic import lilliefors
from statsmodels.distributions.empirical_distribution import ECDF

%matplotlib notebook
data = pd.read_excel("D:\\Data\\Biostat521\\NUTR_521_lab_data.xlsx")
fig_locations = "C:/BrianKim/Code/Projects/biostat521/Lab 1/Figures"

# Organize Data into variables
albumin = data['URXUMS'] # Outcome
HEI = data['HEI2015_TOTAL_SCORE'] # Exposure
gender = data['RIAGENDR']
age = data['RIDAGEYR']
race = data['RIDRETH3']
education = data['DMDEDUC2']
bmi = data['BMXBMI']
systolic = data['BPXSY1']
alc = data['WkAlcDays']
diabetes = data['DIQ010']


# For Plotting Purposes
age_colors = ['red', 'orange', 'yellow', 'green', 'blue', 'purple']
race_colors = ['pink', 'cyan', 'lime', 'tan', 'coral', 'slategray']
edu_colors = ['lavender', 'skyblue', 'palegreen', 'lightcoral', 'wheat']

# Demographics

In [4]:
# Basic Metrics
print("Number of Participants")
print(len(albumin))

# Age Range
print("Age Range")
print(max(age))
print(min(age))

# F/M Count
print("Genders")
print(gender)
type(gender)
gender_counts = gender.value_counts()
print(gender_counts)

Number of Participants
3893
Age Range
80
20
Genders
0         Male
1         Male
2         Male
3       Female
4       Female
         ...  
3888      Male
3889    Female
3890      Male
3891    Female
3892    Female
Name: RIAGENDR, Length: 3893, dtype: object
RIAGENDR
Male      2064
Female    1829
Name: count, dtype: int64


## Univariate Analysis

In [5]:
fig, axs = plt.subplots(2, 2, figsize=(15, 10))

# Age Distribution
bin_edges = [20, 30, 40, 50, 60, 70, 80]
for i in range(len(age_colors)):
    axs[0, 0].hist(
        age,
        bins=[bin_edges[i], bin_edges[i+1]],
        color=age_colors[i],
        alpha=0.6,  # Semi-transparent
        label=f"{bin_edges[i]}-{bin_edges[i+1]}"
    )
axs[0, 0].set_xlabel('Age')
axs[0, 0].set_ylabel('Frequency')
axs[0, 0].set_title('Age Distribution')
axs[0, 0].legend(title='Age Brackets')
axs[0, 0].text(-0.1, 1.05, 'a', transform=axs[0, 0].transAxes, 
            size=16, weight='bold')

# QQ Plot for Uniformity
normalized_age = (age - np.min(age)) / (np.max(age) - np.min(age))
stats.probplot(normalized_age, dist='uniform', plot=axs[0, 1])
axs[0, 1].set_title('QQ Plot for Uniformity')
axs[0, 1].text(-0.1, 1.05, 'b', transform=axs[0, 1].transAxes, 
            size=16, weight='bold')

# Self-Reported Race
unique_races, counts = np.unique(race, return_counts=True)
sorted_idx = np.argsort(counts)[::-1]
sorted_counts = counts[sorted_idx]
sorted_races = unique_races[sorted_idx]
axs[1, 0].bar(sorted_races, sorted_counts, color=race_colors[:len(sorted_races)])
axs[1, 0].set_xlabel('Race')
axs[1, 0].set_ylabel('Count')
axs[1, 0].set_title('Self Reported Race')
axs[1, 0].text(-0.1, 1.05, 'c', transform=axs[1, 0].transAxes, 
            size=16, weight='bold')

# Education Level
unique_edu, counts = np.unique(education, return_counts=True)
custom_order = [2, 4, 1, 3, 0]
custom_counts = counts[custom_order]
custom_edu = unique_edu[custom_order]
bars = axs[1, 1].bar(custom_edu, custom_counts, tick_label=custom_edu, color=edu_colors)
axs[1, 1].set_xlabel('Education Level')
axs[1, 1].set_ylabel('Count')
axs[1, 1].set_title('Education Levels')
edu_percentages = (custom_counts / np.sum(custom_counts)) * 100
for bar, percentage in zip(bars, edu_percentages):
    axs[1, 1].text(bar.get_x() + bar.get_width()/2., bar.get_height(),
             f'{percentage:.2f}%', ha='center', va='bottom')
axs[1, 1].text(-0.1, 1.05, 'd', transform=axs[1, 1].transAxes, 
            size=16, weight='bold')


# Adjust layout
plt.tight_layout()
plt.savefig('Figures/Demographics.png', format='png', dpi=1200)
plt.show()

<IPython.core.display.Javascript object>

In [6]:
colors_gender = ['blue', 'pink']  # blue for Male, pink for Female

# Create a 2x2 grid of subplots
fig, axs = plt.subplots(2, 2, figsize=(15, 10))

# a. Age Distribution
bin_edges = [20, 30, 40, 50, 60, 70, 80]
labels = ['Male', 'Female']
hist_data = [age[gender == 'Male'], age[gender == 'Female']]
colors_gender = ['#1565c0', '#f48fb1']  # Dark blue for Male, light pink for Female

axs[0, 0].hist(hist_data, bins=bin_edges, color=colors_gender, alpha=0.6, label=labels, stacked=True)
axs[0, 0].set_xlabel('Age')
axs[0, 0].set_ylabel('Frequency')
axs[0, 0].set_title('a. Age Distribution')
axs[0, 0].legend()
axs[0, 0].text(-0.1, 1.05, 'a', transform=axs[0, 0].transAxes, 
            size=16, weight='bold')

# b. Age Distribution
male_ages = age[gender == 'Male']
female_ages = age[gender == 'Female']
t_stat, p_val = ttest_ind(male_ages, female_ages)

box_age = [age[gender == 'Male'], age[gender == 'Female']]
labels = ['Male', 'Female']
colors_gender = ['#1565c0', '#f48fb1']  # Dark blue for Male, light pink for Female

axs[0, 1].boxplot(box_age, vert=True, patch_artist=True, labels=labels)
axs[0, 1].text(-0.1, 1.05, 'b', transform=axs[0, 1].transAxes, 
            size=16, weight='bold')

# Check for Significance
if p_val < 0.05:
    # position asterisk roughly in the middle of the two boxes
    axs[0, 1].text(1.5, max(np.concatenate([male_ages, female_ages])) + 1, "*", 
                   ha='center', va='center', fontsize=20, color='red')
    
    # optional: add the p-value itself
    axs[0, 1].text(1.5, max(np.concatenate([male_ages, female_ages])) + 3, 
                   f"p = {p_val:.3f}", 
                   ha='center', va='center', fontsize=12, color='black')

# Applying colors
for patch, color in zip(axs[0, 1].artists, colors_gender):
    patch.set_facecolor(color)

axs[0, 1].set_xlabel('Gender')
axs[0, 1].set_ylabel('Age')
axs[0, 1].set_title('a. Age Distribution by Gender')

# c. Self-Reported Race
total_counts = counts_male + counts_female

# Obtain the indices that would sort the total counts from highest to lowest
sorted_indices = np.argsort(total_counts)[::-1]

# Reorder race labels, male counts, and female counts based on the sorted indices
unique_races = unique_races[sorted_indices]
counts_male = counts_male[sorted_indices]
counts_female = counts_female[sorted_indices]

# Continue with your existing code to plot
width = 0.35
ind = np.arange(len(unique_races))
axs[1, 0].bar(ind - width/2, counts_male, width, label='Male', color='blue')
axs[1, 0].bar(ind + width/2, counts_female, width, label='Female', color='pink')
axs[1, 0].set_xticks(ind)
axs[1, 0].set_xticklabels(unique_races)
axs[1, 0].set_xlabel('Race')
axs[1, 0].set_ylabel('Count')
axs[1, 0].set_title('c. Self Reported Race')
axs[1, 0].legend()
axs[1, 0].text(-0.1, 1.05, 'c', transform=axs[1, 0].transAxes, 
            size=16, weight='bold')

# d. Education Level
custom_order = [2, 4, 1, 3, 0]

unique_edu, counts_male = np.unique(education[gender == 'Male'], return_counts=True)
_, counts_female = np.unique(education[gender == 'Female'], return_counts=True)
# Reorder based on custom order
custom_edu = unique_edu[custom_order]
counts_male = counts_male[custom_order]
counts_female = counts_female[custom_order]

# Continue with your existing code to plot
ind = np.arange(len(custom_edu))
axs[1, 1].bar(ind - width/2, counts_male, width, label='Male', color='blue')
axs[1, 1].bar(ind + width/2, counts_female, width, label='Female', color='pink')
axs[1, 1].set_xticks(ind)
axs[1, 1].set_xticklabels(custom_edu)
axs[1, 1].set_xlabel('Education Level')
axs[1, 1].set_ylabel('Count')
axs[1, 1].set_title('d. Education Levels')
axs[1, 1].legend()
axs[1, 1].text(-0.1, 1.05, 'd', transform=axs[1, 1].transAxes, 
            size=16, weight='bold')
plt.tight_layout()
plt.savefig('Figures/DemographicsGender.png', format='png', dpi=1200)
plt.show()

<IPython.core.display.Javascript object>

NameError: name 'counts_male' is not defined

In [7]:
type(box_age)
male_ages = age[gender == 'Male']
female_ages = age[gender == 'Female']
t_stat, p_val = ttest_ind(male_ages, female_ages)
print(f"T-statistic: {t_stat}")
print(f"P-value: {p_val}")

alpha = 0.05
if p_val < alpha:
    print("The age distributions for males and females are significantly different.")
else:
    print("The age distributions for males and females are not significantly different.")

T-statistic: 2.4060457001845488
P-value: 0.016172599473636907
The age distributions for males and females are significantly different.


## Health Metrics

In [8]:
# Health
fig, axs = plt.subplots(2, 2, figsize=(15, 10))

# a. BMI
axs[0, 0].hist(bmi, bins=50)
axs[0, 0].set_xlabel('BMI')
axs[0, 0].set_ylabel('Occurrence')
axs[0, 0].set_title('Body Mass Index')
axs[0, 0].text(-0.1, 1.05, 'a', transform=axs[0, 0].transAxes, 
            size=16, weight='bold')




# b. Number of Alcoholic Drinks / Week
axs[0, 1].hist(alc, bins=50)
axs[0, 1].set_xlabel('Number of Alcoholic Drinks/Week')
axs[0, 1].set_ylabel('Occurrence (log)')
axs[0, 1].set_yscale('log')
axs[0, 1].set_title('Alcoholic Drinks per Week')
axs[0, 1].text(-0.1, 1.05, 'b', transform=axs[0, 1].transAxes, 
            size=16, weight='bold')

# Systolic Blood Pressure
axs[1, 0].hist(systolic, bins=50)
axs[1, 0].set_xlabel('Systolic Blood Pressure')
axs[1, 0].set_ylabel('Occurrence')
axs[1, 0].set_title('Systolic Blood Pressure')
axs[1, 0].text(-0.1, 1.05, 'c', transform=axs[1, 0].transAxes, 
            size=16, weight='bold')
# Diabetes
diabetes = diabetes.replace("Borderlinerline", "Borderline")
diabetes_counts = diabetes.value_counts().sort_index().values
diabetes_labels = diabetes.value_counts().sort_index().index

axs[1, 1].bar(diabetes_labels, diabetes_counts)
axs[1, 1].set_xlabel('Diabetes')
axs[1, 1].set_ylabel('Count (log)')
axs[1, 1].set_yscale('log')
axs[1, 1].set_title('Diabetes Status')
axs[1, 1].text(-0.1, 1.05, 'd', transform=axs[1, 1].transAxes, 
            size=16, weight='bold')

plt.savefig('Figures/Health.png', format='png', dpi=1200)
plt.show()

<IPython.core.display.Javascript object>

In [9]:
bmi_mean = np.nanmean(bmi)
bmi_median = np.nanmedian(bmi)
bmi_sd = np.nanstd(bmi, ddof=1)  # use ddof=1 to get sample std dev
print(f'Mean BMI: {bmi_mean}\nMedian BMI: {bmi_median}\nStandard Deviation: {bmi_sd}')

systolic_mean = np.nanmean(systolic)
systolic_median = np.nanmedian(systolic)
systolic_sd = np.nanstd(systolic, ddof=1)  # use ddof=1 to get sample std dev
print(f'Mean Systolic: {systolic_mean}\nMedian Systolic: {systolic_median}\nStandard Deviation: {systolic_sd}')

Mean BMI: 29.728608380755304
Median BMI: 28.6
Standard Deviation: 7.099584301973247
Mean Systolic: 125.75026680896478
Median Systolic: 124.0
Standard Deviation: 18.317298345110338


In [10]:
# Albumin
fig, axs = plt.subplots(2, 2, figsize=(15, 10))



# a. 
axs[0, 0].text(-0.1, 1.05, 'a', transform=axs[0, 0].transAxes, 
            size=16, weight='bold')
axs[0, 0].hist(albumin, bins = 50)
axs[0, 0].set_yscale('log')
axs[0, 0].set_xlabel('Albumin Levels (mg/L)')
axs[0, 0].set_ylabel('Occurrences (log)')
axs[0, 0].set_title('Histogram of Albumin Levels in dataset')

# b. Box plot of Albumin Levels within 0-2000
filtered_albumin = albumin[(albumin >= 0) & (albumin <= 2000)]
axs[0, 1].text(-0.1, 1.05, 'b', transform=axs[0, 1].transAxes, 
            size=16, weight='bold')
axs[0, 1].boxplot(filtered_albumin)
axs[0, 1].set_ylabel('Albumin Levels (mg/L)')
axs[0, 1].set_title('Box plot of Albumin Levels (0-2000)')

# c.
axs[1, 0].text(-0.1, 1.05, 'c', transform=axs[1, 0].transAxes, 
            size=16, weight='bold')
axs[1, 0].hist(albumin, bins=50, range=(0, 2000))
axs[1, 0].set_xlim(0, 2000)
axs[1, 0].set_yscale("log")  # Convert the y-axis to a log scale
axs[1, 0].set_xlabel('Albumin Levels (mg/L)')
axs[1, 0].set_ylabel('Occurrences (log)')
axs[1, 0].set_title('Histogram of Albumin Levels in dataset')

# d.
axs[1, 1].text(-0.1, 1.05, 'd', transform=axs[1, 1].transAxes, 
            size=16, weight='bold')
axs[1, 1].hist(albumin, bins=50, range=(2000, 10000))
axs[1, 1].set_xlim(2000, 10000)
axs[1, 1].set_ylim(0, 5)
axs[1, 1].set_xlabel('Albumin Levels (mg/L)')
axs[1, 1].set_ylabel('Occurrences')
axs[1, 1].set_title('Histogram of Albumin Levels in dataset')

plt.tight_layout()
plt.savefig('Figures/Albumin.png', format='png', dpi=1200)

plt.show()

<IPython.core.display.Javascript object>

As can be seen here, there are several outliers (potentially categorized as values above 2000) that obscue the trend of the dataset. 


In [11]:
# HEI 2015
fig, axs = plt.subplots(2, 2, figsize=(15, 10))
HEI_mean = np.mean(HEI)
HEI_median = np.median(HEI)
HEI_sd = np.std(HEI, ddof=1)  # use ddof=1 to get sample std dev

# a. Univariate Data Visualization
axs[0, 0].text(-0.1, 1.05, 'a', transform=axs[0, 0].transAxes, 
            size=16, weight='bold')
axs[0, 0].hist(HEI, bins=50)

axs[0, 0].set_xlabel('HEI Total Score')
axs[0, 0].set_ylabel('Occurrence')
axs[0, 0].set_title('Distribution of HEI Scores')

# b. Checks for Normality
axs[0, 1].text(-0.1, 1.05, 'b', transform=axs[0, 1].transAxes, 
            size=16, weight='bold')
stats.probplot(HEI, dist="norm", plot=axs[0, 1])
axs[0, 1].set_title('QQ-Plot of HEI Scores')
axs[0, 1].set_xlabel('Theoretical Quantiles')
axs[0, 1].set_ylabel('Ordered Values')

# c. Plot Cumulative Distribution Plot
axs[1, 0].text(-0.1, 1.05, 'c', transform=axs[1, 0].transAxes, 
            size=16, weight='bold')
ecdf = ECDF(HEI)
axs[1, 0].plot(ecdf.x, ecdf.y, label='Empirical CDF')
x_values = np.linspace(min(HEI), max(HEI))
axs[1, 0].plot(x_values, stats.norm.cdf(x_values, HEI_mean, HEI_sd), 'r-', label='Standard Normal CDF')
axs[1, 0].legend(loc='best')
axs[1, 0].set_title('Cumulative Distribution Function')
axs[1, 0].set_xlabel('HEI Values')
axs[1, 0].set_ylabel('Cumulative Proportion')

# d. Compare HEI x Albumin
axs[1, 1].text(-0.1, 1.05, 'd', transform=axs[1, 1].transAxes, 
            size=16, weight='bold')
axs[1, 1].scatter(HEI, albumin, alpha = 0.5)
axs[1, 1].set_title('Scatterplot of HEI vs Albumin')
axs[1, 1].set_xlabel('HEI')
axs[1, 1].set_ylabel('Albumin Levels (mg/L)')

plt.savefig('Figures/HEIxAlbumin.png', format='png', dpi=1200)

plt.show()

<IPython.core.display.Javascript object>

In [12]:
# Basic Statistics
## Albumin Outliers
Q1 = np.percentile(albumin, 25)
Q3 = np.percentile(albumin, 75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = albumin[(albumin < lower_bound) | (albumin > upper_bound)]

print(lower_bound)
print(upper_bound)

# Plot Scatterplots based on outlier parameters
fig, axs = plt.subplots(2, 2, figsize=(15, 10))


# Filter data based on the lower and upper bounds for albumin
ofiltered_HEI = HEI[(albumin >= lower_bound) & (albumin <= upper_bound)]
ofiltered_albumin = albumin[(albumin >= lower_bound) & (albumin <= upper_bound)]

# a. Scatter Plot
axs[0, 0].text(-0.1, 1.05, 'a', transform=axs[0, 0].transAxes, 
            size=16, weight='bold')
axs[0, 0].scatter(ofiltered_HEI, ofiltered_albumin, alpha=0.6)
axs[0, 0].set_xlabel('HEI')
axs[0, 0].set_ylabel('Albumin')
axs[0, 0].set_title('Scatterplot of HEI vs. Albumin (Albumin < 38.25)')

slope, intercept = np.polyfit(ofiltered_HEI, ofiltered_albumin, 1)
axs[0, 0].plot(ofiltered_HEI, slope*ofiltered_HEI + intercept, color='red', label=f'Linear regression (y={slope:.2f}x + {intercept:.2f})')
r_value = np.corrcoef(ofiltered_HEI, ofiltered_albumin)[0, 1]

axs[0, 0].text(0.05, 0.95, f'y={slope:.2f}x + {intercept:.2f}', color='red', transform=axs[0, 0].transAxes, verticalalignment='top')
axs[0, 0].text(0.05, 0.90, f'r = {r_value:.2f}', color='red', transform=axs[0, 0].transAxes, verticalalignment='top')

# b. Scatterplot with arbitrary parameters Albumin < 100
# Filter data based on the lower and upper bounds for albumin
filtered_HEI = HEI[(albumin <= 100)]
filtered_albumin = albumin[(albumin <= 100)]

axs[0, 1].text(-0.1, 1.05, 'b', transform=axs[0, 1].transAxes, 
            size=16, weight='bold')
axs[0, 1].scatter(filtered_HEI, filtered_albumin, alpha=0.6)
axs[0, 1].set_xlabel('HEI')
axs[0, 1].set_ylabel('Albumin')
axs[0, 1].set_title('Scatterplot of HEI vs. Albumin (Albumin < 100)')

slope, intercept = np.polyfit(filtered_HEI, filtered_albumin, 1)
axs[0, 1].plot(filtered_HEI, slope*filtered_HEI + intercept, color='red', label=f'Linear regression (y={slope:.2f}x + {intercept:.2f})')
r_value = np.corrcoef(filtered_HEI, filtered_albumin)[0, 1]

axs[0, 1].text(0.05, 0.95, f'y={slope:.2f}x + {intercept:.2f}', color='red', transform=axs[0, 1].transAxes, verticalalignment='top')
axs[0, 1].text(0.05, 0.90, f'r = {r_value:.2f}', color='red', transform=axs[0, 1].transAxes, verticalalignment='top')

# c. Scatterplot with arbitrary parameters Albumin < 500
filtered_HEI = HEI[(albumin <= 500)]
filtered_albumin = albumin[(albumin <= 500)]

axs[1, 0].text(-0.1, 1.05, 'c', transform=axs[1, 0].transAxes, 
            size=16, weight='bold')
axs[1, 0].scatter(filtered_HEI, filtered_albumin, alpha=0.6)
axs[1, 0].set_xlabel('HEI')
axs[1, 0].set_ylabel('Albumin')
axs[1, 0].set_title('Scatterplot of HEI vs. Albumin (Albumin < 500)')

slope, intercept = np.polyfit(filtered_HEI, filtered_albumin, 1)
axs[1, 0].plot(filtered_HEI, slope*filtered_HEI + intercept, color='red', label=f'Linear regression (y={slope:.2f}x + {intercept:.2f})')
r_value = np.corrcoef(filtered_HEI, filtered_albumin)[0, 1]

axs[1, 0].text(0.05, 0.95, f'y={slope:.2f}x + {intercept:.2f}', color='red', transform=axs[1, 0].transAxes, verticalalignment='top')
axs[1, 0].text(0.05, 0.90, f'r = {r_value:.2f}', color='red', transform=axs[1, 0].transAxes, verticalalignment='top')



# d. Scatterplot with arbitrary parameters Albumin < 2000
filtered_HEI = HEI[(albumin <= 2000)]
filtered_albumin = albumin[(albumin <= 2000)]

axs[1, 1].text(-0.1, 1.05, 'd', transform=axs[1, 1].transAxes, 
            size=16, weight='bold')
axs[1, 1].scatter(filtered_HEI, filtered_albumin, alpha=0.6)
axs[1, 1].set_xlabel('HEI')
axs[1, 1].set_ylabel('Albumin')
axs[1, 1].set_title('Scatterplot of HEI vs. Albumin (Albumin < 2000)')

slope, intercept = np.polyfit(filtered_HEI, filtered_albumin, 1)
axs[1, 1].plot(filtered_HEI, slope*filtered_HEI + intercept, color='red', label=f'Linear regression (y={slope:.2f}x + {intercept:.2f})')
r_value = np.corrcoef(filtered_HEI, filtered_albumin)[0, 1]

axs[1, 1].text(0.05, 0.95, f'y={slope:.2f}x + {intercept:.2f}', color='red', transform=axs[1, 1].transAxes, verticalalignment='top')
axs[1, 1].text(0.05, 0.90, f'r = {r_value:.2f}', color='red', transform=axs[1, 1].transAxes, verticalalignment='top')


plt.savefig("Figures/HEIxAlbuminScatter.png", format='png', dpi=1200)
plt.show()




-15.75
38.25


<IPython.core.display.Javascript object>

In [47]:
# HEI x Others
ofiltered_HEI = HEI[(albumin >= lower_bound) & (albumin <= upper_bound)]
ofiltered_albumin = albumin[(albumin >= lower_bound) & (albumin <= upper_bound)]
ofiltered_bmi = bmi[(albumin >= lower_bound) & (albumin <= upper_bound)]
ofiltered_systolic = systolic[(albumin >= lower_bound) & (albumin <= upper_bound)]
ofiltered_alc = alc[(albumin >= lower_bound) & (albumin <= upper_bound)]
ofiltered_diabetes = diabetes[(albumin >= lower_bound) & (albumin <= upper_bound)]

In [33]:

fig, axs = plt.subplots(3, 2, figsize=(15, 15))
## a. HEI x Gender
axs[0, 0].text(-0.1, 1.05, 'a', transform=axs[0, 0].transAxes, 
            size=16, weight='bold')
HEI_male = HEI[gender == 'Male']
HEI_female = HEI[gender == 'Female']

axs[0, 0].hist(HEI_male, bins=20, density=True, color='#1565c0', alpha=0.6, label='Male')
axs[0, 0].hist(HEI_female, bins=20, density=True, color='#f48fb1', alpha=0.6, label='Female')
axs[0, 0].set_title('HEI Distribution by Gender, accounting for n')
axs[0, 0].set_xlabel('HEI')
axs[0, 0].set_ylabel('Frequency')

## b. HEI x Age
axs[0, 1].text(-0.1, 1.05, 'b', transform=axs[0, 1].transAxes, 
            size=16, weight='bold')
axs[0, 1].scatter(age, HEI)
axs[0, 1].set_xlabel('Age')
axs[0, 1].set_ylabel('HEI Score')
axs[0, 1].set_title('Age vs HEI Score')

slope, intercept = np.polyfit(age, HEI, 1)
axs[0, 1].plot(age, slope*age + intercept, color='red', label=f'Linear regression (y={slope:.2f}x + {intercept:.2f})')
r_value = np.corrcoef(age, HEI)[0, 1]

axs[0, 1].text(0.05, 0.95, f'y={slope:.2f}x + {intercept:.2f}', color='red', transform=axs[0, 1].transAxes, verticalalignment='top')
axs[0, 1].text(0.05, 0.90, f'r = {r_value:.2f}', color='red', transform=axs[0, 1].transAxes, verticalalignment='top')

## c. Albumin x HEI x Gender
axs[0, 1].text(-0.1, 1.05, 'c', transform=axs[1, 0].transAxes, 
            size=16, weight='bold')
unique_genders = ['Male', 'Female']
color_map = {'Male': 'blue', 'Female': 'red'}
for gen, color in color_map.items():
    is_gender = gender == gen
    axs[1, 0].scatter(ofiltered_HEI[is_gender], ofiltered_albumin[is_gender], color=color, label=gen, alpha=0.4)
    slope, intercept = np.polyfit(ofiltered_HEI[is_gender], ofiltered_albumin[is_gender], 1)
    axs[1, 0].plot(ofiltered_HEI[is_gender], slope*ofiltered_HEI[is_gender] + intercept, color= color, label=f'Linear regression (y={slope:.2f}x + {intercept:.2f})')
axs[1, 0].set_xlabel('HEI 2015 Total Score')
axs[1, 0].set_ylabel('BMI')
axs[1, 0].set_title('BMI vs. HEI 2015 Total Score colored by Gender')

## d. Albumin x HEI x Race
axs[1, 1].text(-0.1, 1.05, 'd', transform=axs[1, 1].transAxes, 
            size=16, weight='bold')

unique_races = ['White', 'Black', 'MexicanAmerican', 'OtherHispanic', 'Asian', 'Other']
color_map = {unique_race: race_colors[idx] for idx, unique_race in enumerate(unique_races)}

for r, color in color_map.items():
    is_race = race == r
    axs[1, 1].scatter(ofiltered_HEI[is_race], ofiltered_albumin[is_race], color=color, label=r, alpha=0.4)
    slope, intercept = np.polyfit(ofiltered_HEI[is_race], ofiltered_albumin[is_race], 1)
    
    axs[1, 1].plot(ofiltered_HEI[is_race], slope*ofiltered_HEI[is_race] + intercept, color= color, label=f'Linear regression (y={slope:.2f}x + {intercept:.2f})')

axs[1, 1].set_xlabel('HEI')
axs[1, 1].set_ylabel('Albumin mg/L')
axs[1, 1].set_title('Hei vs Albumin categorized by race.')

## e. Albumin x HEI x BMI
axs[2, 0].text(-0.1, 1.05, 'e', transform=axs[1, 0].transAxes, 
            size=16, weight='bold')
def bmi_category(bmi_value):
    if bmi_value < 18.5:
        return "low"
    elif 18.5 <= bmi_value < 25:
        return "medium"
    else:
        return "high"

bmi_clusters = ofiltered_bmi.apply(bmi_category)
color_map = {
    'low': 'blue',
    'medium': 'green',
    'high': 'red'
}
### Plot data for each BMI category
for category, color in color_map.items():
    axs[2, 0].scatter(ofiltered_HEI[bmi_clusters == category], 
                ofiltered_albumin[bmi_clusters == category], 
                c=color, 
                label=category,
                edgecolor='k',
                alpha=0.7)
    slope, intercept = np.polyfit(ofiltered_HEI[bmi_clusters == category], ofiltered_albumin[bmi_clusters == category], 1)
    
    axs[2, 0].plot(ofiltered_HEI[bmi_clusters == category], slope*ofiltered_HEI[bmi_clusters == category] + intercept, color= color, label=f'Linear regression (y={slope:.2f}x + {intercept:.2f})')

axs[2, 0].set_xlabel('HEI')
axs[2, 0].set_ylabel('Albumin mg/L')
axs[2, 0].set_title('HEI vs Albumin categorized by BMI')
axs[2, 0].legend()

## f. Albumin x HEI x systolic
axs[2, 1].text(-0.1, 1.05, 'f', transform=axs[1, 0].transAxes, 
            size=16, weight='bold')
def sbp_category(sbp_value):
    if sbp_value < 90:
        return "low"
    elif 90 <= sbp_value < 140:
        return "normal"
    else:
        return "high"

systolic_clusters = ofiltered_systolic.apply(sbp_category)
color_map = {
    'low': 'blue',
    'normal': 'green',
    'high': 'red'
}

### Plot data for each SBP category
for category, color in color_map.items():
    axs[2, 1].scatter(ofiltered_HEI[systolic_clusters == category], 
                ofiltered_albumin[systolic_clusters == category], 
                c=color, 
                label=category,
                edgecolor='k',
                alpha=0.7)
    slope, intercept = np.polyfit(ofiltered_HEI[systolic_clusters == category], ofiltered_albumin[systolic_clusters == category], 1)
    
    axs[2, 1].plot(ofiltered_HEI[systolic_clusters == category], slope*ofiltered_HEI[systolic_clusters == category] + intercept, color=color, label=f'Linear regression (y={slope:.2f}x + {intercept:.2f})')

axs[2, 1].set_xlabel('HEI')
axs[2, 1].set_ylabel('Albumin mg/L')
axs[2, 1].set_title('HEI vs Albumin categorized by Systolic Blood Pressure')
axs[2, 1].legend()


plt.show()
plt.savefig("Figures/HEIxAlbuminxCategory.png", format='png', dpi=1200)

<IPython.core.display.Javascript object>

In [51]:
# Create a mask where 'ofiltered_systolic' is not NaN
mask = ~np.isnan(ofiltered_systolic)

# Apply the mask to filter out the NaNs
ofiltered_systolic = ofiltered_systolic[mask]
ofiltered_HEI = ofiltered_HEI[mask]
ofiltered_albumin = ofiltered_albumin[mask]
ofiltered_bmi = ofiltered_bmi[mask]

mask = ~np.isnan(ofiltered_bmi)
ofiltered_systolic = ofiltered_systolic[mask]
ofiltered_HEI = ofiltered_HEI[mask]
ofiltered_albumin = ofiltered_albumin[mask]
ofiltered_bmi = ofiltered_bmi[mask]

In [58]:
# Plot Confounds
fig, axs = plt.subplots(2, 2, figsize=(15, 10))
## a. Systolic x HEI
axs[0, 0].text(-0.1, 1.05, 'a', transform=axs[0, 0].transAxes, 
            size=16, weight='bold')
axs[0, 0].scatter(ofiltered_systolic, ofiltered_HEI)
slope, intercept = np.polyfit(ofiltered_systolic, ofiltered_HEI, 1)
axs[0, 0].plot(ofiltered_systolic, slope*ofiltered_systolic + intercept, color=color, label=f'Linear regression (y={slope:.2f}x + {intercept:.2f})')
axs[0, 0].set_xlabel('Systolic Blood Pressure')
axs[0, 0].set_ylabel('HEI')
axs[0, 0].set_title('Systolic Blood Pressure vs. HEI Scores')
axs[0, 0].legend()

## b. Systolic x Albumin
axs[0, 1].text(-0.1, 1.05, 'b', transform=axs[0, 1].transAxes, 
            size=16, weight='bold')
axs[0, 1].scatter(ofiltered_systolic, ofiltered_albumin)
slope, intercept = np.polyfit(ofiltered_systolic, ofiltered_albumin, 1)
axs[0, 1].plot(ofiltered_systolic, slope*ofiltered_systolic + intercept, color=color, label=f'Linear regression (y={slope:.2f}x + {intercept:.2f})')
axs[0, 1].set_xlabel('Systolic Blood Pressure')
axs[0, 1].set_ylabel('Albumin')
axs[0, 1].set_title('Systolic Blood Pressure vs. Albumin Levels')
axs[0, 1].legend()

## c. BMI x HEI
axs[1, 0].text(-0.1, 1.05, 'c', transform=axs[1, 0].transAxes, 
            size=16, weight='bold')
axs[1, 0].scatter(ofiltered_bmi, ofiltered_HEI)
slope, intercept = np.polyfit(ofiltered_bmi, ofiltered_HEI, 1)
axs[1, 0].plot(ofiltered_bmi, slope*ofiltered_bmi + intercept, color=color, label=f'Linear regression (y={slope:.2f}x + {intercept:.2f})')
axs[1, 0].set_xlabel('BMI')
axs[1, 0].set_ylabel('HEI')
axs[1, 0].set_title('BMI vs HEI Scores')
axs[1, 0].legend()

## d. BMI x Albumin
axs[1, 1].text(-0.1, 1.05, 'c', transform=axs[1, 1].transAxes, 
            size=16, weight='bold')
axs[1, 1].scatter(ofiltered_bmi, ofiltered_albumin)
slope, intercept = np.polyfit(ofiltered_bmi, ofiltered_albumin, 1)
axs[1, 1].plot(ofiltered_bmi, slope*ofiltered_bmi + intercept, color=color, label=f'Linear regression (y={slope:.2f}x + {intercept:.2f})')
axs[1, 1].set_xlabel('BMI')
axs[1, 1].set_ylabel('HEI')
axs[1, 1].set_title('BMI vs Albumin Levels')
axs[1, 1].legend()

plt.show()
plt.savefig("Figures/HEIAlbuminxSBPBMI.png", format='png', dpi=1200)

<IPython.core.display.Javascript object>

In [52]:

plt.figure()
plt.scatter(ofiltered_bmi, ofiltered_HEI)
slope, intercept = np.polyfit(ofiltered_bmi, ofiltered_HEI, 1)
plt.plot(ofiltered_bmi, slope*ofiltered_bmi + intercept, color=color, label=f'Linear regression (y={slope:.2f}x + {intercept:.2f})')
plt.xlabel('BMI')
plt.ylabel('HEI')
plt.legend()

plt.figure()
plt.scatter(ofiltered_bmi, ofiltered_albumin)
slope, intercept = np.polyfit(ofiltered_bmi, ofiltered_albumin, 1)
plt.plot(ofiltered_bmi, slope*ofiltered_bmi + intercept, color=color, label=f'Linear regression (y={slope:.2f}x + {intercept:.2f})')
plt.xlabel('BMI')
plt.ylabel('Albumin')
plt.legend()


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [44]:
print(ofiltered_bmi)

0       27.8
1       30.8
3       42.4
5       28.0
6       28.2
        ... 
3887    22.2
3888    32.0
3889    28.1
3890    32.9
3892    21.4
Name: BMXBMI, Length: 3433, dtype: float64


In [16]:
plt.figure()
unique_edu = ['NoHighSchool', 'SomeHighSchool', 'HighSchool', 'SomeCollege', 'College']
color_map = {unique_race: edu_colors[idx] for idx, unique_race in enumerate(unique_edu)}

for edu, color in color_map.items():
    is_edu = education == edu
    plt.scatter(ofiltered_HEI[is_edu], ofiltered_albumin[is_edu], color=color, label=edu, alpha=0.4)
    slope, intercept = np.polyfit(ofiltered_HEI[is_edu], ofiltered_albumin[is_edu], 1)
    plt.plot(ofiltered_HEI[is_edu], slope*ofiltered_HEI[is_edu] + intercept, color= color, label=f'Linear regression (y={slope:.2f}x + {intercept:.2f})')
plt.xlabel('HEI')
plt.ylabel('Albumin mg/L')
plt.title('Hei vs Albumin categorized by education level.')
plt.legend()

<IPython.core.display.Javascript object>

In [17]:
import statsmodels.api as sm
data = {
    'HEI': ofiltered_HEI,
    'albumin': ofiltered_albumin,
    'education': education
}
df = pd.DataFrame(data)

# Convert the 'Education' column into dummy variables
edu_dummies = pd.get_dummies(df['education'])  # drop_first prevents multicollinearity
df = pd.concat([df, edu_dummies], axis=1)
df.drop('education', axis=1, inplace=True)  # drop original 'Education' column
df['HEI'] = df['HEI'].astype(float)
df['albumin'] = df['albumin'].astype(float)
df.dropna(inplace=True)
df[['HighSchool', 'NoHighSchool', 'SomeCollege', 'SomeHighSchool', 'College']] = df[['HighSchool', 'NoHighSchool', 'SomeCollege', 'SomeHighSchool', 'College']].astype(int)
# Define predictors and the outcome variable
X = df.drop('albumin', axis=1)  # predictors
X = sm.add_constant(X)  # add a constant (intercept)
y = df['albumin']  # outcome variable

# Perform regression
model = sm.OLS(y, X).fit()

# Print regression results
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                albumin   R-squared:                       0.016
Model:                            OLS   Adj. R-squared:                  0.014
Method:                 Least Squares   F-statistic:                     10.88
Date:                Sun, 01 Oct 2023   Prob (F-statistic):           2.10e-10
Time:                        18:00:13   Log-Likelihood:                -12027.
No. Observations:                3433   AIC:                         2.407e+04
Df Residuals:                    3427   BIC:                         2.410e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const             10.5645      0.439     24.

In [62]:
print(df.dtypes)

HEI               float64
albumin           float64
HighSchool           bool
NoHighSchool         bool
SomeCollege          bool
SomeHighSchool       bool
dtype: object
